In [ ]:
# import xgboost
# print("xgboost", xgboost.__version__)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier

In [33]:
data=pd.read_csv('card_transactions_all_features.csv')
data.head()

,Recnum,Cardnum,Date,Merchnum,Merchdescription,Merchstate,Merchzip,Transtype,Amount,Fraud,...,card_state_mean_030,card_state_total_030,card_state_mean_13,card_state_total_13,card_state_mean_17,card_state_total_17,card_state_mean_114,card_state_total_114,card_state_mean_130,card_state_total_130
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,...,30.0,30.0,3.0,3.0,7.0,7.0,14.0,14.0,30.0,30.0


In [34]:
##exclude test data and few initial columns
data_mod=data[(data.Date>'2010-01-14')].reset_index(drop=True)

In [35]:
data_mod.drop(columns=['Recnum','Cardnum','Merchnum','Merchdescription','Merchstate','Merchzip',\
                     'Transtype','Amount','card_merchant','card_zip','card_state','dow_risk'],inplace=True)

In [36]:
train_test=data_mod[data_mod.Date<'2010-11-01']
fraud_train_test=train_test['Fraud']
train_test.drop(columns=['Date','Fraud'],inplace=True)
OOT=data_mod[data_mod.Date>='2010-11-01']
fraud_oot=OOT['Fraud']
OOT.drop(columns=['Date','Fraud'],inplace=True)


/Users/sharathreddy/Programs/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [37]:
train_test=train_test.fillna(0)
OOT=OOT.fillna(0)

In [38]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
train_test_zs = sc.fit_transform(train_test)
OOT_zs=sc.transform(OOT)

In [39]:
train_test_zs_final = pd.DataFrame(train_test_zs, columns = train_test.columns)
train_test_zs_final['Fraud']=fraud_train_test
# train_test_zs_final=train_test_zs_final.fillna(0)
OOT_zs_final = pd.DataFrame(OOT_zs, columns = OOT.columns)
OOT_zs_final['Fraud']=fraud_oot
# OOT_zs_final=OOT_zs_final.fillna(0)

In [40]:
print(f'train test zs final has {train_test_zs_final.shape[0]} rows {train_test_zs_final.shape[1]} columns \
    \n OOT zs final has {OOT_zs_final.shape[0]}rows and {OOT_zs_final.shape[1]}columns')

train test zs final has 80632 rows 396 columns     
 OOT zs final has 12427rows and 396columns


## Incorporating dataframe with only selected top 30 vars

In [41]:
top_30_vars=pd.read_csv('Copy of top_30_vars_final.csv')

In [42]:
ord_model_vars=top_30_vars[['sno_variable','var_name']]

In [43]:
X_trntst=train_test_zs_final.loc[:,ord_model_vars['var_name']]
Y_trntst=fraud_train_test
OOT_final_x=OOT_zs_final.loc[:,ord_model_vars['var_name']]
OOT_final_y=fraud_oot

In [44]:
def fdr(classifier,x,y,cutoff=0.03):
  '''calculates fdr for given classifier on dataset x and y with cutoff values'''
  return fdr_prob(y,classifier.predict_proba(x),cutoff)

def fdr_prob(y,y_prob,cutoff=0.03):
  """calculates FDR score using a prob list of predicted y"""
  if len(y_prob.shape)!=1:
    y_prob=y_prob[:,-1:]
  num_fraud=len(y[y==1])
  sorted_prob=np.asarray(sorted(zip(y_prob,y),key= lambda x:x[0],reverse=True))
  cutoff_bin=sorted_prob[0:int(len(y)*cutoff),1:] ##3% cutoff
  return len(cutoff_bin[cutoff_bin==1])/num_fraud

In [1]:
# from sklearn.model_selection import KFold, GridSearchCV

# search_space = [
#   {
#     'n_estimators': [50, 100, 150, 200,500,1000],
#     'learning_rate': [0.001,0.01, 0.1, 0.2, 0.3],
#     'max_depth': range(3, 10),
#     'colsample_bytree': [i/10.0 for i in range(1, 3)],
#     'gamma': [i/10.0 for i in range(3)]
#   }
# ]
# kfold = KFold(n_splits=10, random_state=42)
# grid = GridSearchCV(
#   XGBClassifier(objective='binary:logistic'),
#   param_grid=search_space,
#   cv=kfold,
#   scoring=fdr,
#   verbose=1,
#   n_jobs=-1
# )
# # Fit grid search
# model = grid.fit(X_trntst,Y_trntst )

In [2]:
# best_pars = grid.best_params_

In [3]:
# best_pars

In [4]:
# X_trntst

In [27]:
# predictions=model.predict_proba(X_trntst.iloc[:,:32])[:,1]

In [28]:
# # predictions=model.predict_proba(X_trntst)[:,1]
# # FDR3=pd.DataFrame()
# X_trntst['predicted']=predictions
# X_trntst['Fraud']=Y_trntst
# topRows=int(round(X_trntst.shape[0]*0.03))
# temp=X_trntst.sort_values('predicted',ascending=False).head(topRows)
# needed=temp.loc[:,'Fraud']
# trn_fdr=sum(needed)/sum(X_trntst.loc[:,'Fraud'])


# predictions=model.predict_proba(OOT_final_x)[:,1]

# OOT_final_x['predicted']=predictions
# OOT_final_x['Fraud']=OOT_final_y.values
# #         print(X_oot_eval)
# topRows=int(round(OOT_final_x.shape[0]*0.03))
# temp=OOT_final_x.sort_values('predicted',ascending=False).head(topRows)
# #         print(temp)
# needed=temp.loc[:,'Fraud']
# oot_fdr=sum(needed)/sum(OOT_final_x.loc[:,'Fraud'])

In [5]:
# oot_fdr

In [6]:
# trn_fdr

In [ ]:
weight
scale_pos_weight=weight,

In [155]:
def avg_iter_xgb(iterations,learning_rate,weight,estimators,depth,train_set,test_set,oot_x,oot_y):
    FDR3=pd.DataFrame()
    nitermax=iterations
    X_trntst=train_set
    Y_trntst=test_set
    OOT_final_x=oot_x
    OOT_final_y=oot_y
    
    for niter in range(nitermax):
        X_trn,X_tst,Y_trn,Y_tst=train_test_split(X_trntst,Y_trntst, test_size=0.20)

        model=XGBClassifier(learning_rate=learning_rate,scale_pos_weight=weight,n_estimators=estimators,max_depth=depth,\
                           objective= 'binary:logistic',eval_metric='logloss')

        X_trn_array=X_trn.values
        X_tst_array=X_tst.values
        X_oot_array=OOT_final_x.values
        Y_trn_array=Y_trn.values
        Y_tst_array=Y_tst.values
        Y_oot_array=OOT_final_y

        X_oot=OOT_final_x.copy()
        X_trn_eval=X_trn.copy()
        X_tst_eval=X_tst.copy()
        X_oot_eval=X_oot.copy()

        model.fit(X_trn,Y_trn)
        predictions=model.predict_proba(X_trn)[:,1]

        X_trn_eval['predicted']=predictions
        X_trn_eval['Fraud']=Y_trn
        topRows=int(round(X_trn_eval.shape[0]*0.03))
        temp=X_trn_eval.sort_values('predicted',ascending=False).head(topRows)
        needed=temp.loc[:,'Fraud']
        FDR3.loc[niter,'trn']=sum(needed)/sum(X_trn_eval.loc[:,'Fraud'])

        predictions=model.predict_proba(X_tst)[:,1]

        X_tst_eval['predicted']=predictions
        X_tst_eval['Fraud']=Y_tst
        topRows=int(round(X_tst_eval.shape[0]*0.03))
        temp=X_tst_eval.sort_values('predicted',ascending=False).head(topRows)
        needed=temp.loc[:,'Fraud']
        FDR3.loc[niter,'tst']=sum(needed)/sum(X_tst_eval.loc[:,'Fraud'])

        predictions=model.predict_proba(X_oot)[:,1]

        X_oot_eval['predicted']=predictions
        X_oot_eval['Fraud']=OOT_final_y.values
#         print(X_oot_eval)
        topRows=int(round(X_oot_eval.shape[0]*0.03))
        temp=X_oot_eval.sort_values('predicted',ascending=False).head(topRows)
#         print(temp)
        needed=temp.loc[:,'Fraud']
        FDR3.loc[niter,'oot']=sum(needed)/sum(X_oot_eval.loc[:,'Fraud'])
#         print(FDR3.loc[niter,'oot'])
        
    return list([FDR3.loc[:,'trn'].mean(),FDR3.loc[:,'tst'].mean(),FDR3.loc[:,'oot'].mean()])
#         print(niter,FDR3.loc[niter,'trn'],FDR3.loc[niter,'tst'],FDR3.loc[niter,'oot'])
    

In [153]:
features=[10,20,30]
max_depth=[3,6,10]
weight=len(data.loc[data['Fraud']==0,['Fraud']])/len(data.loc[data['Fraud']==1,['Fraud']])
learning_rate =[0.1,0.01,0.001]
estimators=[500,600,800,900,1000,1100]


In [15]:
results_xgb={}
count=0
for num in features:
    results_xgb[num]=(avg_iter_xgb(10,0.1,weight,500,3,X_trntst.iloc[:,:num],Y_trntst,OOT_final_x.iloc[:,:num],OOT_final_y))
    count+=1
    print(count)
    results_xgb[num].append(avg_iter_xgb(10,0.1,weight,600,3,X_trntst.iloc[:,:num],Y_trntst,OOT_final_x.iloc[:,:num],OOT_final_y))
    count+=1
    print(count)
    results_xgb[num].append(avg_iter_xgb(10,0.1,weight,800,6,X_trntst.iloc[:,:num],Y_trntst,OOT_final_x.iloc[:,:num],OOT_final_y))
    count+=1
    print(count)
    results_xgb[num].append(avg_iter_xgb(10,0.01,weight,900,6,X_trntst.iloc[:,:num],Y_trntst,OOT_final_x.iloc[:,:num],OOT_final_y))
    count+=1
    print(count)
    results_xgb[num].append(avg_iter_xgb(10,0.001,weight,1000,6,X_trntst.iloc[:,:num],Y_trntst,OOT_final_x.iloc[:,:num],OOT_final_y))
    count+=1
    print(count)
    results_xgb[num].append(avg_iter_xgb(10,0.001,weight,1000,8,X_trntst.iloc[:,:num],Y_trntst,OOT_final_x.iloc[:,:num],OOT_final_y))
    count+=1
    print(count)
    results_xgb[num].append(avg_iter_xgb(10,0.1,weight,1000,10,X_trntst.iloc[:,:num],Y_trntst,OOT_final_x.iloc[:,:num],OOT_final_y))
    count+=1
    print(count)
    results_xgb[num].append(avg_iter_xgb(10,0.1,weight,900,6,X_trntst.iloc[:,:num],Y_trntst,OOT_final_x.iloc[:,:num],OOT_final_y))
    count+=1
    print(count)
    
    
    
    

/Users/sharathreddy/Programs/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1176: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/sharathreddy/Programs/anaconda3/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [16]:
results_xgb

{10: [0.995068786813758,
  0.8833861876528504,
  0.18268156424581003,
  [0.9952759544960269, 0.8869126756016203, 0.1776536312849162],
  [1.0, 0.845661649679446, 0.15698324022346372],
  [0.9913097027529696, 0.8893391759514314, 0.1793296089385475],
  [0.9054396360303004, 0.808470952540268, 0.24804469273743018],
  [0.9583174373927651, 0.8022279584181385, 0.253072625698324],
  [1.0, 0.8219864542683858, 0.15642458100558662],
  [1.0, 0.8374805579694, 0.15586592178770953]],
 20: [0.9955040472426919,
  0.8824831358738632,
  0.2011173184357542,
  [0.9957468900231289, 0.8869262380717856, 0.18715083798882678],
  [1.0, 0.862394524809772, 0.16759776536312848],
  [0.9960374816109144, 0.8746150223011007, 0.2659217877094972],
  [0.8982895679553812, 0.8015345325524492, 0.3100558659217877],
  [0.9623457019914327, 0.8059946478294213, 0.27709497206703915],
  [1.0, 0.8351954095335007, 0.18379888268156422],
  [1.0, 0.8560095912994516, 0.18603351955307265]],
 30: [0.9950854283678636,
  0.8866900633424374,
  

In [24]:
new_res[30]=(avg_iter_xgb(10,0.001,weight,1100,6,X_trntst.iloc[:,:30],Y_trntst,OOT_final_x.iloc[:,:30],OOT_final_y))
print(4)
print(new_res)
new_res[30].append(avg_iter_xgb(10,0.001,99,1200,6,X_trntst.iloc[:,:30],Y_trntst,OOT_final_x.iloc[:,:30],OOT_final_y))
print(5)
print(new_res)
new_res[30].append(avg_iter_xgb(10,0.001,99,1300,6,X_trntst.iloc[:,:30],Y_trntst,OOT_final_x.iloc[:,:30],OOT_final_y))
print(6)
print(new_res)


4
{30: [0.9109111469044198, 0.8062065944201112, 0.32178770949720664]}
5
{30: [0.9109111469044198, 0.8062065944201112, 0.32178770949720664, [0.9203974297660921, 0.8002452858032425, 0.3463687150837989]]}
6
{30: [0.9109111469044198, 0.8062065944201112, 0.32178770949720664, [0.9203974297660921, 0.8002452858032425, 0.3463687150837989], [0.923031362497628, 0.8054298214892215, 0.32346368715083795]]}


In [25]:
new_res[30].append(avg_iter_xgb(10,0.001,99,1400,6,X_trntst.iloc[:,:30],Y_trntst,OOT_final_x.iloc[:,:30],OOT_final_y))
print(7)
print(new_res)
new_res[30].append(avg_iter_xgb(10,0.0001,99,1200,7,X_trntst.iloc[:,:30],Y_trntst,OOT_final_x.iloc[:,:30],OOT_final_y))
print(8)
print(new_res)
new_res[30].append(avg_iter_xgb(10,0.0001,99,1250,8,X_trntst.iloc[:,:30],Y_trntst,OOT_final_x.iloc[:,:30],OOT_final_y))
print(9)
print(new_res)

7
{30: [0.9109111469044198, 0.8062065944201112, 0.32178770949720664, [0.9203974297660921, 0.8002452858032425, 0.3463687150837989], [0.923031362497628, 0.8054298214892215, 0.32346368715083795], [0.924734667575204, 0.8056038945410646, 0.3050279329608938]]}
8
{30: [0.9109111469044198, 0.8062065944201112, 0.32178770949720664, [0.9203974297660921, 0.8002452858032425, 0.3463687150837989], [0.923031362497628, 0.8054298214892215, 0.32346368715083795], [0.924734667575204, 0.8056038945410646, 0.3050279329608938], [0.875528251794597, 0.7526809176175411, 0.375977653631285]]}
9
{30: [0.9109111469044198, 0.8062065944201112, 0.32178770949720664, [0.9203974297660921, 0.8002452858032425, 0.3463687150837989], [0.923031362497628, 0.8054298214892215, 0.32346368715083795], [0.924734667575204, 0.8056038945410646, 0.3050279329608938], [0.875528251794597, 0.7526809176175411, 0.375977653631285], [0.9095771976933886, 0.765601236089408, 0.293854748603352]]}


In [26]:
new_res[30].append(avg_iter_xgb(10,0.0001,101,1200,8,X_trntst.iloc[:,:30],Y_trntst,OOT_final_x.iloc[:,:30],OOT_final_y))
print(9)
print(new_res)

9
{30: [0.9109111469044198, 0.8062065944201112, 0.32178770949720664, [0.9203974297660921, 0.8002452858032425, 0.3463687150837989], [0.923031362497628, 0.8054298214892215, 0.32346368715083795], [0.924734667575204, 0.8056038945410646, 0.3050279329608938], [0.875528251794597, 0.7526809176175411, 0.375977653631285], [0.9095771976933886, 0.765601236089408, 0.293854748603352], [0.9083013373810646, 0.7602499063274137, 0.3]]}


In [28]:
new_res[25]=(avg_iter_xgb(10,0.0001,99,1200,8,X_trntst.iloc[:,:25],Y_trntst,OOT_final_x.iloc[:,:25],OOT_final_y))
print(10)
print(new_res)

10
{30: [0.9109111469044198, 0.8062065944201112, 0.32178770949720664, [0.9203974297660921, 0.8002452858032425, 0.3463687150837989], [0.923031362497628, 0.8054298214892215, 0.32346368715083795], [0.924734667575204, 0.8056038945410646, 0.3050279329608938], [0.875528251794597, 0.7526809176175411, 0.375977653631285], [0.9095771976933886, 0.765601236089408, 0.293854748603352], [0.9083013373810646, 0.7602499063274137, 0.3]], 25: [0.9158842875919226, 0.7684064138692246, 0.2849162011173184]}


### Adabooost model

In [3]:
FDR3=pd.DataFrame()
def avg_iter_adab(iterations,learning_rate,estimators,depth,train_set,test_set,oot_x,oot_y):
    
    nitermax=iterations
    X_trntst=train_set
    Y_trntst=test_set
    OOT_final_x=oot_x
    OOT_final_y=oot_y
    
    for niter in range(nitermax):
        X_trn,X_tst,Y_trn,Y_tst=train_test_split(X_trntst,Y_trntst, test_size=0.3)

        model=AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth=depth),\
                                 learning_rate=learning_rate,\
                                 n_estimators=estimators)

        X_trn_array=X_trn.values
        X_tst_array=X_tst.values
        X_oot_array=OOT_final_x.values
        Y_trn_array=Y_trn.values
        Y_tst_array=Y_tst.values
        Y_oot_array=OOT_final_y

        X_oot=OOT_final_x.copy()
        X_trn_eval=X_trn.copy()
        X_tst_eval=X_tst.copy()
        X_oot_eval=X_oot.copy()

        model.fit(X_trn,Y_trn)
        predictions=model.predict_proba(X_trn)[:,1]

        X_trn_eval['predicted']=predictions
        X_trn_eval['Fraud']=Y_trn
        topRows=int(round(X_trn_eval.shape[0]*0.03))
        temp=X_trn_eval.sort_values('predicted',ascending=False).head(topRows)
        needed=temp.loc[:,'Fraud']
        FDR3.loc[niter,'trn']=sum(needed)/sum(X_trn_eval.loc[:,'Fraud'])

        predictions=model.predict_proba(X_tst)[:,1]

        X_tst_eval['predicted']=predictions
        X_tst_eval['Fraud']=Y_tst
        topRows=int(round(X_tst_eval.shape[0]*0.03))
        temp=X_tst_eval.sort_values('predicted',ascending=False).head(topRows)
        needed=temp.loc[:,'Fraud']
        FDR3.loc[niter,'tst']=sum(needed)/sum(X_tst_eval.loc[:,'Fraud'])

        predictions=model.predict_proba(X_oot)[:,1]

        X_oot_eval['predicted']=predictions
        X_oot_eval['Fraud']=OOT_final_y.values
#         print(X_oot_eval)
        topRows=int(round(X_oot_eval.shape[0]*0.03))
        temp=X_oot_eval.sort_values('predicted',ascending=False).head(topRows)
#         print(temp)
        needed=temp.loc[:,'Fraud']
        FDR3.loc[niter,'oot']=sum(needed)/sum(X_oot_eval.loc[:,'Fraud'])
#         print(FDR3.loc[niter,'oot'])
        
    return list([FDR3.loc[:,'trn'].mean(),FDR3.loc[:,'tst'].mean(),FDR3.loc[:,'oot'].mean()])
#         print(niter,FDR3.loc[niter,'trn'],FDR3.loc[niter,'tst'],FDR3.loc[niter,'oot'])
    

In [ ]:
results_adab={}
count=0
for num in features:
    results_adab[num]=(avg_iter_adab(10,0.1,100,3,X_trntst.iloc[:,:num],Y_trntst,OOT_final_x.iloc[:,:num],OOT_final_y))
    count+=1
    print(count)
    results_adab[num].append(avg_iter_adab(10,0.1,200,3,X_trntst.iloc[:,:num],Y_trntst,OOT_final_x.iloc[:,:num],OOT_final_y))
    count+=1
    print(count)
    results_adab[num].append(avg_iter_adab(10,0.1,250,6,X_trntst.iloc[:,:num],Y_trntst,OOT_final_x.iloc[:,:num],OOT_final_y))
    count+=1
    print(count)
    results_adab[num].append(avg_iter_adab(10,0.001,300,6,X_trntst.iloc[:,:num],Y_trntst,OOT_final_x.iloc[:,:num],OOT_final_y))
    count+=1
    print(count)
    results_adab[num].append(avg_iter_adab(10,0.001,300,6,X_trntst.iloc[:,:num],Y_trntst,OOT_final_x.iloc[:,:num],OOT_final_y))
    count+=1
    print(count)
    results_adab[num].append(avg_iter_adab(10,0.1,400,10,X_trntst.iloc[:,:num],Y_trntst,OOT_final_x.iloc[:,:num],OOT_final_y))
    count+=1
    print(count)
    results_adab[num].append(avg_iter_adab(10,0.1,500,6,X_trntst.iloc[:,:num],Y_trntst,OOT_final_x.iloc[:,:num],OOT_final_y))
    count+=1
    print(count)

In [ ]:
results_adab={}
results_adab[30]=(avg_iter_adab(10,0.01,1000,6,X_trntst.iloc[:,:30],Y_trntst,OOT_final_x.iloc[:,:30],OOT_final_y))
print(1)
print(results_adab)
results_adab[25]=(avg_iter_adab(10,0.001,1200,7,X_trntst.iloc[:,:25],Y_trntst,OOT_final_x.iloc[:,:25],OOT_final_y))
print(2)
print(results_adab)
results_adab[20]=(avg_iter_adab(10,0.001,1200,30,X_trntst.iloc[:,:20],Y_trntst,OOT_final_x.iloc[:,:20],OOT_final_y))
print(3)
print(results_adab)


In [ ]:
results_adab[30].append(avg_iter_adab(10,0.001,1200,6,X_trntst.iloc[:,:30],Y_trntst,OOT_final_x.iloc[:,:30],OOT_final_y))
print(6)
print(results_adab)